# Data Evaluation

The purpose of this notebook is to streamline the process of manually evaluating the extracted test case data from notebook 1.

For this we will sample test cases from the data (using a fixed seed for reproducibility) and have information about the test cases be displayed for manual evaluation, including the relevant lines of code. 

## Imports

In [1]:
import pandas as pd
import os

## Setup

In [2]:
# Set library root folder:

# For TensorFlow
library_root_tensorflow = "A:/BachelorThesis/DLL_Testing_Tool/DL_Libraries/Tensorflow/tensorflow-master/tensorflow/python/" 
tests_root_tensorflow = "kernel_tests"
save_data_to_tensorflow = "extracted_data/tensorflow_evaluation_data.csv"

# For Pytorch
library_root_pytorch = "A:/BachelorThesis/DLL_Testing_Tool/DL_Libraries/PyTorch/pytorch-master/" 
tests_root_pytorch = "test"
save_data_to_pytorch = "extracted_data/pytorch_evaluation_data.csv"


## Import the data

In [3]:
# import tensorflow data and preview
df_tensorflow = pd.read_csv('extracted_data/tensorflow_data.csv')
df_tensorflow.head(10)

,File_Path,Line_Number,Found_in_Function,Function_Definition_Line_Number,Assert_Statement_Type,Oracle_Argument_ Position,Differential_Function_Line_Number,Differential_Test_Function
0,kernel_tests\aggregate_ops_test.py,69,testAddN,59,assertAllClose,1,66,np.sum
1,kernel_tests\aggregate_ops_test.py,83,testUnknownShapes,72,assertAllClose,1,80,np.sum
2,kernel_tests\aggregate_ops_test.py,83,testUnknownShapes,72,assertAllClose,2,79,sess.run
3,kernel_tests\array_ops_test.py,172,CheckVersusNumpy,150,assertAllClose,1,159,NaN
4,kernel_tests\array_ops_test.py,172,CheckVersusNumpy,150,assertAllClose,1,161,NaN
5,kernel_tests\array_ops_test.py,172,CheckVersusNumpy,150,assertAllClose,1,163,NaN
6,kernel_tests\array_ops_test.py,172,CheckVersusNumpy,150,assertAllClose,2,164,array_ops.boolean_mask
7,kernel_tests\array_ops_test.py,217,testEmptyInput2D,210,assertAllClose,1,213,NaN
8,kernel_tests\array_ops_test.py,217,testEmptyInput2D,210,assertAllClose,2,214,array_ops.boolean_mask
9,kernel_tests\array_ops_test.py,226,testEmptyInput1D,219,assertAllClose,1,222,NaN


In [4]:
# import pytorch data and preview
df_pytorch = pd.read_csv('extracted_data/pytorch_data.csv')
df_pytorch.head(10)

,File_Path,Line_Number,Found_in_Function,Function_Definition_Line_Number,Assert_Statement_Type,Oracle_Argument_ Position,Differential_Function_Line_Number,Differential_Test_Function
0,test\test_ao_sparse.py,98,test_sparse_qlinear,32,assert_array_almost_equal,1,96,dense_qlinear_dynamic
1,test\test_ao_sparse.py,98,test_sparse_qlinear,32,assert_array_almost_equal,2,95,sparse_qlinear_dynamic
2,test\test_ao_sparse.py,103,test_sparse_qlinear,32,assert_array_almost_equal,1,101,dense_qlinear
3,test\test_ao_sparse.py,103,test_sparse_qlinear,32,assert_array_almost_equal,2,100,sparse_qlinear
4,test\test_autograd.py,1329,test_set_grad_coroutines_benign_exceptions,1295,assertLess,1,1328,coro.throw
5,test\test_autograd.py,1329,test_set_grad_coroutines_benign_exceptions,1295,assertLess,1,1339,coro.throw
6,test\test_autograd.py,1340,test_set_grad_coroutines_benign_exceptions,1295,assertLess,1,1328,coro.throw
7,test\test_autograd.py,1340,test_set_grad_coroutines_benign_exceptions,1295,assertLess,1,1339,coro.throw
8,test\test_autograd.py,2790,test_var_mean_differentiable,2778,allclose,1,2788,UNSUPPORTED Binary Operation
9,test\test_autograd.py,2790,test_var_mean_differentiable,2778,allclose,2,2789,UNSUPPORTED Binary Operation


## Analyze coverage

To track the progress of our test case extraction we display statistics about how many cases still are still unsupported. This is either denoted by an "UNSUPPORTED ..." statement in the "Differential Test Function" column of the data or by an empty string in this column, i.e. `NaN`.

In [5]:
not_covered_tensorflow = df_tensorflow[df_tensorflow['Differential_Test_Function'].str.contains('UNSUPPORTED', na=False) | df_tensorflow['Differential_Test_Function'].isna()]
print("TensorFlow:\t"+ str(len(not_covered_tensorflow)) + " cases not covered.")

not_covered_pytorch = df_pytorch[df_pytorch['Differential_Test_Function'].str.contains('UNSUPPORTED', na=False) | df_pytorch['Differential_Test_Function'].isna()]
print("PyTorch:\t"+ str(len(not_covered_pytorch)) + " cases not covered.")

TensorFlow:	320 cases not covered.
PyTorch:	157 cases not covered.


# Tool for manual evaluation

This tool is meant to help with quickly evaluating test cases from the dataset. For each test case, it prints all information collected about the case, including the oracle argument position and the extracted function name, as well as the code inside the function where the test case was defined. Then the evaluator is asked for an evaluation of the test case via input. This evaluation is then stored alongside the test case in the data.

Evaluation keys:  
y: Test case correctly identified  
n: Test case is not differential testing  
?: Allows for the entry of a comment. This is meant for situations where the current case is differential testing, but the differential testing function was not extracted correctly (or some other data is incorrect).  

In [13]:
class EvaluationAutomator:
    def __init__(self, df, library_root, save_data_to):
        """Initialize the evaluation automator.
        
        df: Dataframe to evaluate.
        library_root: The root folder of the DL library
        save_data_to: Relative location to load/save the evaluation data
        """
        self.df = df
        self.save_data_to = save_data_to
        self.library_root = library_root
        
        # try importing evaluation data if it already exists
        if os.path.isfile(self.save_data_to): 
            self.eval_df = pd.read_csv(self.save_data_to)
            print("Evaluation data opened.")
        
        # otherwise initialize evaluation df and add new column for the evaluation result
        else:
            self.eval_df = df.copy()
            todo_list = ["TODO"] * len(self.eval_df.index)
            self.eval_df.insert(len(df.columns), 'Evaluation', todo_list)
            self.eval_df.to_csv(self.save_data_to)
            print("New evaluation data created.")
            
            
    def evaluate(self, index):
        """Present the data entry at the given index for evaluation."""
        
        # present the data entry
        print(self.df.iloc[index])
        print("\n")
        
        # check if it has already been evaluated
        if self.eval_df.at[index, 'Evaluation'] != "TODO":
            print("Already evaluated! Previous evaluation: " + self.eval_df.at[index, 'Evaluation'])
            if input("Re-evaluate? (y / n) ") != "y":
                return
            
        
        # print the relevant source code lines:
        
        # get source file of current test case and open it as an array of lines
        source = open(self.library_root + self.df.iloc[index]['File_Path']).readlines()

        # set beginning and end line number for the code section to display
        beginning_line_no = self.df.iloc[index]['Function_Definition_Line_Number']
        end_line_no = self.df.iloc[index]['Line_Number']

        # print these lines
        for line in range(beginning_line_no, end_line_no+1):
            print(str(line) + ": " + source[line-1])
        
        # ask for a decision from the evaluator:
        decision_bool = True
        while decision_bool:
            decision = input("Correctly identified? (y / n / ?): ")
            
            if decision in ["y", "n"]:
                decision_bool = False

            elif decision == "?":
                decision = input("Please comment on this case: ")
                decision_bool = False
                
            else:
                print("Error. Please specify y/n/?")
                decision_bool = True
                
        # write the decision to the evaluation data
        self.eval_df.at[index, 'Evaluation'] = decision
        self.eval_df.to_csv(self.save_data_to, index=False)

# initialize automator:
# TensorFlow
evalAutomator_tensorflow = EvaluationAutomator(df_tensorflow, library_root_tensorflow, save_data_to_tensorflow)

# PyTorch
evalAutomator_pytorch = EvaluationAutomator(df_pytorch, library_root_pytorch, save_data_to_pytorch)

# test evaluation on a particular case
evalAutomator_tensorflow.evaluate(2)

Evaluation data opened.
Evaluation data opened.
File_Path                            kernel_tests\aggregate_ops_test.py
Line_Number                                                          83
Found_in_Function                                     testUnknownShapes
Function_Definition_Line_Number                                      72
Assert_Statement_Type                                    assertAllClose
Oracle_Argument_ Position                                             2
Differential_Function_Line_Number                                    79
Differential_Test_Function                                     sess.run
Name: 2, dtype: object


72:   def testUnknownShapes(self):

73:     np.random.seed(12345)

74:     with self.session() as sess:

75:       for dtype in self._supported_types():

76:         data = self._buildData((2, 2), dtype)

77:         for count in range(1, self._MAX_N + 1):

78:           data_ph = array_ops.placeholder(dtype=dtype)

79:           actual = sess.run(m

## Sampling cases for evaluation
TODO